In [18]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# This bot was written by Sam Reeves.  For all questions or comments
# please email me.  samtreeves@gmail.com

import keys
import dalekBrain
import random
import tweepy
import time
import os.path
import dill
import dalekBrain


class dalek(object):
    "I am a twitter bot. Create an instance of me with bot = dalek()"

    def __init__(self):
        self.auth = tweepy.OAuthHandler(
                keys.consumer_key, keys.consumer_secret)
        self.auth.set_access_token(
                keys.access_token, keys.access_token_secret)

        self.api = tweepy.API(self.auth)
        if os.path.exists('local_cache.pkl'):
            f = open("local_cache.pkl", "rb")
            self.cache = dill.load(f)
        else:
            self.cache = {}

    def limitHandled(cursor):
        while True:
            try:
                yield cursor.next()
            except tweepy.RateLimitError:
                time.sleep(15*60)
                
    def writeCache(self):
        f = open("local_cache.pkl", "wb")
        dill.dump(self.cache, f)
        f.close()
        
    def updateFollowers(self):
        for follower in tweepy.Cursor(self.api.followers).items():
            if follower.id not in self.cache['followers']:
                self.cache['followers'].append(follower.id)

    def updateFriends(self):
        for friend in tweepy.Cursor(self.api.friends).items():
            if friend.id not in self.cache['friends']:
                self.cache['friends'].append(friend.id)

    def updateDMs(self):
        # do a call for new dms
        new_dm_call = self.api.list_direct_messages()
        
        # if no cache exists, create one
        if 'dm_cache' not in self.cache:
            for dm in new_dm_call:
                self.cache['dm_cache'][int(dm.id)] = dm
            self.cache['most_recent'] = max(self.cache['most_recent'])

        # if cache exists, update with unique data
        elif max(self.cache['dm_cache']) <= max(new_dm_call):
            for dm in new_dm_call:
                if self.cache['most_recent'] < dm.id:
                    self.cache['dm_cache'][dm.id] = dm
                    
    def followEveryoneBack(self):
        for follower_id in self.cache['followers']:
            if follower_id not in self.cache['friends']:
                self.api.follow(follower_id)

    def addUser(self, user_id, admin_status=False):
        # if users not cached, create empty list
        if 'users' not in self.cache:
            self.cache['users'] = {'user' : [], 'admin' : []}
            
        # add user
        self.cache['users']['user'].append(user_id)
        
        # add admin rights
        if admin_status:
            self.cache['users']['admin'].append()
            
        # remove duplicates
        self.cache['users']['user'] = set(self.cache['users']['user'])
        self.cache['users']['admin'] = set(self.cache['users']['admin'])

    def removeUser(self, user_id, wipe=False):
        
        # if users not cached, create empty list
        if 'users' not in self.cache:
            self.cache['users'] = [[], []]
        
        #if user is admin, remove admin
        if user_id in self.cache['users'][1]:
            self.cache['users'][1].remove(user_id)
        
        #also remove admin from users
            if wipe:
                self.cache['users'][0].remove(user_id)
        
        #remove non-admin user
        elif user_id in self.cache['users'][0]:
            self.cache['users'][0].remove(user_id)

    def addStatus(status):
        self.cache['statuses'].append(status)

    def groupConversations(self):
        # if no hash exists, create one
        if 'conversations' not in self.cache:
            self.cache['conversations'] = {}
            
        # check every cached message to see if it is in a conversation
        for message_id in self.cache['dm_cache'].keys():
            user_id = int(self.cache['dm_cache'][message_id].message_create['sender_id'])
            
            # begin a conversation
            if user_id not in self.cache['conversations'].keys():
                self.cache['conversations'][user_id] = []

            # continue a conversation
            if message_id not in self.cache['conversations'].values():
                self.cache['conversations'][user_id].append(message_id)


In [19]:
bot = dalek()

In [35]:
bot.cache['dm_cache'][1229869491045257223].message_create['message_data']['entities']['hashtags'][0]['text']

'spicy'